In [1]:
from PIL import Image
from skimage import io, transform

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision as tv

outputs = [
    "Dots",
    "Paisley",
    "Qua",
    "Square",
    "Strips",
    "Traingles"
]


# nn model using pytorch schematic
class PatternRecognitionNet(nn.Module):

    def __init__(self):
        super(PatternRecognitionNet, self).__init__()

        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(16 * 73 * 73, 120) 
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 6)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
def getImageTensor():
    
    # loading image from local memory
    path = "/home/krys/POLIBUDA/Patterns_database/test_image/"
    image_name = input()
    
    i_path = path + image_name
    image = io.imread(i_path)
    
    return image
    

# transforms image to grayscale tensor
def myTransform(image):

    transformedImage =  tv.transforms.Compose([
                        tv.transforms.ToPILImage(),
                        tv.transforms.Grayscale(num_output_channels=1),
                        tv.transforms.ToTensor()
                      ])(image)
    
    transformedImage = transformedImage[0]

    return transformedImage

# create parrter recognition model 

PATH = "/home/krys/POLIBUDA/Patterns_database/model.pt"

pattNet = PatternRecognitionNet()
pattNet.load_state_dict(torch.load(PATH))
pattNet.eval()

PatternRecognitionNet(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=85264, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=6, bias=True)
)

In [2]:
image = getImageTensor()
t_image = myTransform(image)
input = t_image.view(-1,1,300,300)
output = pattNet(input)
print(output)
val, idx = output[0].max(0)

print("\n")
print("Your predicted output is:", outputs[idx])

tr54.jpg
tensor([[ 0.1096, -0.0074,  0.0766,  0.0657, -0.0219,  0.7740]],
       grad_fn=<AddmmBackward>)


Your predicted output is: Traingles
